<a href="https://colab.research.google.com/github/ChavesHygor/BairesDev---Machine-Learning-Training-ok/blob/main/Reconhecimento_facial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
# --- ETAPA 1: INSTALAÇÃO DAS BIBLIOTECAS ---

# Instala as bibliotecas necessárias. Usamos 'headless' para o OpenCV em ambientes sem GUI como o Colab.
print("Instalando bibliotecas: TensorFlow, OpenCV, MTCNN e Scikit-learn...")
!pip install -q tensorflow opencv-python-headless mtcnn scikit-learn
print("Instalação concluída!")

Instalando bibliotecas: TensorFlow, OpenCV, MTCNN e Scikit-learn...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 36.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 64.3 MB/s eta 0:00:00
Instalação concluída!


In [7]:
# --- ETAPA 2: CONFIGURAÇÃO DO GOOGLE DRIVE E PASTAS ---
import os
from google.colab import drive

# Monta o Google Drive
print("Montando o Google Drive...")
drive.mount('/content/gdrive')

# Define o caminho base para o nosso projeto
base_dir = '/content/gdrive/My Drive/facial_recognition'

# Cria a estrutura de pastas
dataset_dir = os.path.join(base_dir, 'dataset')
output_dir = os.path.join(base_dir, 'output')

os.makedirs(dataset_dir, exist_ok=True)
os.makedirs(output_dir, exist_ok=True)

print(f"Estrutura de pastas criada em: '{base_dir}'")
print("\nINSTRUÇÃO IMPORTANTE:")
print(f"Agora, acesse a pasta '{dataset_dir}' no seu Google Drive.")
print("Dentro dela, crie uma pasta para cada pessoa que você quer reconhecer (ex: 'Albert Einstein', 'Marie Curie').")
print("Coloque várias fotos (5 a 10) de cada pessoa em sua respectiva pasta.")

Montando o Google Drive...
Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
Estrutura de pastas criada em: '/content/gdrive/My Drive/facial_recognition'

INSTRUÇÃO IMPORTANTE:
Agora, acesse a pasta '/content/gdrive/My Drive/facial_recognition/dataset' no seu Google Drive.
Dentro dela, crie uma pasta para cada pessoa que você quer reconhecer (ex: 'Albert Einstein', 'Marie Curie').
Coloque várias fotos (5 a 10) de cada pessoa em sua respectiva pasta.


In [8]:
# --- ETAPA 3: DOWNLOAD E VERIFICAÇÃO DO MODELO FACENET (COM O LINK DO REPOSITÓRIO INDICADO) ---
import os

# Define os caminhos necessários
base_dir = '/content/gdrive/My Drive/facial_recognition'
output_dir = os.path.join(base_dir, 'output')
os.makedirs(output_dir, exist_ok=True) # Garante que a pasta de saída existe
facenet_model_path = os.path.join(output_dir, 'facenet_keras.h5')

# 1. Deleta qualquer versão antiga/corrompida do arquivo
print("Removendo qualquer arquivo de modelo antigo para garantir um download limpo...")
!rm -f "{facenet_model_path}"

# 2. USA O LINK DE DOWNLOAD DIRETO DO REPOSITÓRIO GITHUB 'nyoki-mtl/keras-facenet'
print("\nBaixando o modelo FaceNet do repositório 'nyoki-mtl' (aprox. 95MB)...")
!wget --show-progress -O "{facenet_model_path}" "https://github.com/D2KLab/Face-Celebrity-Recognition/raw/master/model/facenet_keras.h5"

# 3. VERIFICA O SUCESSO DO DOWNLOAD
print("\n--- Verificação Pós-Download ---")
# Verifica se o arquivo existe e tem um tamanho razoável (mais de 1MB)
if os.path.exists(facenet_model_path) and os.path.getsize(facenet_model_path) > 1000000:
    # 3A. Verifica o tamanho do arquivo (deve ser em torno de 95MB)
    print("Verificando o tamanho do arquivo...")
    !ls -lh "{facenet_model_path}"

    # 3B. Verifica o tipo do arquivo (deve ser 'Hierarchical Data Format (HDF) data')
    print("\nVerificando o tipo do arquivo...")
    !file "{facenet_model_path}"
    print("\nDOWNLOAD BEM-SUCEDIDO! Se o tamanho for ~95M e o tipo for 'HDF data', você pode prosseguir para a Célula 4.")
else:
    print("\nERRO CRÍTICO: O download falhou. O arquivo não foi criado ou está vazio.")
    print("Verifique sua conexão ou se o novo link ainda é válido.")


Removendo qualquer arquivo de modelo antigo para garantir um download limpo...

Baixando o modelo FaceNet do repositório 'nyoki-mtl' (aprox. 95MB)...
--2025-08-26 23:16:57--  https://github.com/D2KLab/Face-Celebrity-Recognition/raw/master/model/facenet_keras.h5
Resolving github.com (github.com)... 140.82.121.3
Connecting to github.com (github.com)|140.82.121.3|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://github.com/D2KLab/FaceRec/raw/master/model/facenet_keras.h5 [following]
--2025-08-26 23:16:57--  https://github.com/D2KLab/FaceRec/raw/master/model/facenet_keras.h5
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/D2KLab/FaceRec/master/model/facenet_keras.h5 [following]
--2025-08-26 23:16:57--  https://raw.githubusercontent.com/D2KLab/FaceRec/master/model/facenet_keras.h5
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.

In [9]:
# --- ETAPA 4: PROCESSAMENTO DO DATASET E GERAÇÃO DE EMBEDDINGS (VERSÃO MELHORADA) ---
import os
import cv2
import numpy as np
from mtcnn.mtcnn import MTCNN
from tensorflow.keras.models import load_model

# Define o caminho para o modelo e VERIFICA SE ELE EXISTE
facenet_model_path = os.path.join(output_dir, 'facenet_keras.h5')
if not os.path.exists(facenet_model_path):
    print(f"ERRO CRÍTICO: O arquivo do modelo FaceNet não foi encontrado em '{facenet_model_path}'")
    print("Por favor, execute a Célula 3 para fazer o download do modelo antes de continuar.")
else:
    print("Iniciando a geração de embeddings...")

    # Carrega os modelos
    detector = MTCNN()
    facenet_model = load_model(facenet_model_path)
    print("Modelos MTCNN e FaceNet carregados.")

    def get_embedding(face_pixels):
        # Padroniza os valores dos pixels
        face_pixels = face_pixels.astype('float32')
        mean, std = face_pixels.mean(), face_pixels.std()
        face_pixels = (face_pixels - mean) / std

        # Adiciona uma dimensão para o lote (batch)
        samples = np.expand_dims(face_pixels, axis=0)

        # Gera o embedding
        yhat = facenet_model.predict(samples)
        return yhat[0]

    # Listas para armazenar os embeddings e os rótulos
    embeddings_list = []
    labels_list = []

    # Itera sobre as pastas de cada pessoa no dataset
    for person_name in os.listdir(dataset_dir):
        person_dir = os.path.join(dataset_dir, person_name)
        if not os.path.isdir(person_dir):
            continue

        print(f"\nProcessando imagens de: {person_name}")
        for image_name in os.listdir(person_dir):
            image_path = os.path.join(person_dir, image_name)
            image = cv2.imread(image_path)

            # Converte para RGB, pois MTCNN espera RGB
            image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

            # Detecta faces
            results = detector.detect_faces(image_rgb)

            # Extrai a face (assume uma face por imagem para o dataset)
            if results:
                x1, y1, width, height = results[0]['box']
                x1, y1 = abs(x1), abs(y1)
                x2, y2 = x1 + width, y1 + height

                face = image_rgb[y1:y2, x1:x2]

                # Redimensiona para o tamanho esperado pelo FaceNet
                face_resized = cv2.resize(face, (160, 160))

                # Gera e armazena o embedding
                embedding = get_embedding(face_resized)
                embeddings_list.append(embedding)
                labels_list.append(person_name)
                print(f"> {image_name}")

    # Salva os embeddings e rótulos
    embeddings_file = os.path.join(output_dir, 'embeddings.npz')
    np.savez_compressed(embeddings_file, embeddings=np.asarray(embeddings_list), labels=np.asarray(labels_list))

    print(f"\nProcesso concluído! Embeddings e rótulos salvos em '{embeddings_file}'")

Iniciando a geração de embeddings...


EOFError: EOF read where object expected

In [10]:
# --- ETAPA 5: TREINAMENTO DO CLASSIFICADOR SVM ---
import numpy as np
import pickle
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC

print("Iniciando o treinamento do classificador...")

# Carrega os embeddings salvos
embeddings_file = os.path.join(output_dir, 'embeddings.npz')
data = np.load(embeddings_file)
embeddings, labels = data['embeddings'], data['labels']

# Codifica os rótulos (nomes) em números
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)

# Treina o modelo SVM
svm_model = SVC(kernel='linear', probability=True)
svm_model.fit(embeddings, encoded_labels)
print("Treinamento do SVM concluído.")

# Salva o modelo treinado e o codificador de rótulos
svm_model_path = os.path.join(output_dir, 'svm_model.pkl')
label_encoder_path = os.path.join(output_dir, 'label_encoder.pkl')

with open(svm_model_path, 'wb') as f:
    pickle.dump(svm_model, f)
with open(label_encoder_path, 'wb') as f:
    pickle.dump(label_encoder, f)

print(f"Modelo SVM salvo em: '{svm_model_path}'")
print(f"Codificador de rótulos salvo em: '{label_encoder_path}'")

Iniciando o treinamento do classificador...


FileNotFoundError: [Errno 2] No such file or directory: '/content/gdrive/My Drive/facial_recognition/output/embeddings.npz'

In [ ]:
# --- ETAPA 6: APLICAÇÃO DE RECONHECIMENTO FACIAL ---
import os
import cv2
import numpy as np
import pickle
from mtcnn.mtcnn import MTCNN
from tensorflow.keras.models import load_model
from google.colab import files
from IPython.display import display, Image

# Carrega todos os modelos e arquivos necessários
print("Carregando modelos e dados...")
detector = MTCNN()
facenet_model = load_model(os.path.join(output_dir, 'facenet_keras.h5'))
svm_model = pickle.load(open(os.path.join(output_dir, 'svm_model.pkl'), 'rb'))
label_encoder = pickle.load(open(os.path.join(output_dir, 'label_encoder.pkl'), 'rb'))
print("Pronto para o reconhecimento.")

# Função para fazer o upload da imagem
uploaded = files.upload()
test_image_path = list(uploaded.keys())[0]

# Carrega e processa a imagem de teste
image = cv2.imread(test_image_path)
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
results = detector.detect_faces(image_rgb)

# Itera sobre todas as faces detectadas na imagem
for result in results:
    x1, y1, width, height = result['box']
    x1, y1 = abs(x1), abs(y1)
    x2, y2 = x1 + width, y1 + height

    face = image_rgb[y1:y2, x1:x2]
    face_resized = cv2.resize(face, (160, 160))

    # Gera o embedding da face detectada
    embedding = get_embedding(face_resized)
    embedding = np.expand_dims(embedding, axis=0)

    # Faz a predição usando o SVM
    prediction = svm_model.predict(embedding)
    proba = svm_model.predict_proba(embedding)
    confidence = np.max(proba)

    # Decodifica o rótulo para obter o nome
    predicted_label = label_encoder.inverse_transform(prediction)[0]

    # Define um limiar de confiança
    if confidence > 0.75: # Limiar de 75% de confiança
        text = f"{predicted_label} ({confidence:.2f})"
    else:
        text = "Desconhecido"

    # Desenha o retângulo e o texto na imagem original
    cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
    cv2.putText(image, text, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

# Salva e exibe a imagem resultante
output_image_path = '/content/output_image.jpg'
cv2.imwrite(output_image_path, image)

print("\n--- Resultado do Reconhecimento ---")
display(Image(output_image_path))